In [1]:
# import処理は時間が掛かるため事前に終わらせておくと良い
import sys
import os
import time
import socket
# wlan ipv4表示
hostname = socket.gethostname()
print(f'wlan IPv4:\"{socket.gethostbyname(hostname)}\"')

os.chdir(os.path.dirname(os.path.abspath("__file__")))
sys.path.append("./recognize_image")
sys.path.append("./manage_signal")

from recognize_image.treasureAreaDetect import TreasureAreaDetect
from recognize_image.minifig import Minifig
from manage_signal.main import MainSignal
from manage_signal.competitionSystem import CompetitionSystem

class Main:
    """
    様々なタスク(スレッドを動作させる)
    ｲﾝｽﾀﾝｽ：競技システム(シングルトンクラス)
    タスク：トレジャーエリアの検知
    タスク：ミニフィグの推測処理
    タスク：IoT列車攻略
    タスク：送受信処理
    """
    def __init__(self):
        self.__competitionSystem = CompetitionSystem()
        # ↓コースの指定
        self.__competitionSystem.setCourse('r')

        # スレッドの初期化
        cameraDevice = 0
        self.__treasureAreaDetect = TreasureAreaDetect(100, cameraDevice)
        self.__minifig = Minifig(100)
        self.__mainSignal = MainSignal()
        print("[ main ]初期化完了")

    def execute(self):
        # スレッドの起動
        self.__treasureAreaDetect.startThread()
        self.__minifig.startThread()
        self.__mainSignal.startThread()

        while True:
            command = input("[ main ]コマンドの入力")

            if self.process_command(command) == True:
                break

    def process_command(self, command):
        isEnd = False

        if command.lower() in ["e", "exit"]:
            self.terminate()
            isEnd = True
        elif command.lower() == "c0":
            self.__treasureAreaDetect.changeCamera(0)
            print("[ main ]0番カメラ切り替え")
        elif command.lower() == "c1":
            self.__treasureAreaDetect.changeCamera(1)
            print("[ main ]1番カメラ切り替え")
        elif command.lower() == "c2":
            self.__treasureAreaDetect.changeCamera(2)
            print("[ main ]2番カメラ切り替え")
        elif command.lower() == "wait":
            time.sleep(1000)
        else:
            print("[ main ]無効なコマンドです")
            
        return isEnd

    def terminate(self):
        self.__treasureAreaDetect.stopThread()
        self.__minifig.stopThread()
        self.__mainSignal.stopThread()

        del self.__treasureAreaDetect
        del self.__minifig
        del self.__mainSignal
        del self.__competitionSystem 
        print("[ main ]プログラムの終了")

wlan IPv4:"10.17.8.181"


In [2]:
### 各種デバイスセッティングができたら押す

## 準備期間でやること
# ①画面にカメラデバイスが起動し検知済みかどうかを確認できるので、カメラの向き調整を調整してください
# ②走行体との通信関係ができているかを確認(通信相手のIPアドレスが表示される)

# 一度プログラムを走らせたらカーネルを再起動推奨
if __name__ == "__main__":
    main = Main()
    main.execute()

[ 競技システム ] rコース用のipv4 "192.168.11.13"を設定しました。
受信用サーバ起動
送信用サーバ起動
Server started : 0.0.0.0:8080
[ main ]初期化完了
Server started : 0.0.0.0:8081
[ TreasureAreaDetect ]以下のブロック配置の決定がされました。
[decoy, treasure, none, none, ]
[none, decoy, none, none, ]
[none, none, none, none, ]
[none, none, none, none, ]
------------
[ TreasureAreaDetect ]走行体のルート生成完了
